In [101]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
!pip install gensim
import gensim
from gensim.models import Word2Vec
import numpy as np


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [102]:
data = pd.read_csv('Liar_Dataset.csv')
data.columns
data.head(10)
data.tail(10)


,[ID].json,label,statement,subject(s),speaker,speaker's job title,state info,party affiliation,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,venue
12777,1271.json,mostly-true,"The proposed excise tax on ""Cadillac"" health p...","health-care,taxes",sarah-palin,NaN,Alaska,republican,9,19,9,6,6,a note on Facebook
12778,278.json,FALSE,"I supported (the surge), I argued for it. I'm ...",iraq,john-mccain,U.S. senator,Arizona,republican,31,39,31,37,8,"a debate in Myrtle Beach, S.C."
12779,7013.json,barely-true,Says U.S. Rep. Charles Bass wants to privatize...,social-security,ann-mclane-kuster,Attorney,New Hampshire,democrat,2,1,3,0,0,"an ad, “Janice,” released September 6, 2012"
12780,8716.json,half-true,More Americans believe the moon landing was fa...,health-care,tom-graves,NaN,NaN,republican,1,0,5,0,1,a Tweet
12781,2661.json,pants-fire,"In the past two years, Democrats have spent mo...","federal-budget,history",eric-cantor,House Majority Leader,Virginia,republican,9,6,4,4,4,an interview on Comedy Central's Daily Show wi...
12782,3419.json,half-true,"For the first time in more than a decade, impo...","energy,oil-spill,trade",barack-obama,President,Illinois,democrat,70,71,160,163,9,a press conference
12783,12548.json,mostly-true,Says Donald Trump has bankrupted his companies...,candidates-biography,hillary-clinton,Presidential candidate,New York,democrat,40,29,69,76,7,a speech on the economy
12784,401.json,TRUE,"John McCain and George Bush have ""absolutely n...",health-care,campaign-defend-america,NaN,"Washington, D.C.",none,0,1,0,2,0,a television ad
12785,1055.json,FALSE,A new poll shows 62 percent support the presid...,health-care,americans-united-change,NaN,NaN,none,1,4,4,1,0,an Internet ad.
12786,9117.json,barely-true,No one claims the report vindicating New Jerse...,"candidates-biography,infrastructure",rudy-giuliani,Attorney,New York,republican,9,11,10,7,3,"comments on NBC's ""Meet the Press"""


In [103]:
data.drop(['[ID].json'], inplace = True, axis = 1)
len(data)

12787

In [104]:
data.drop_duplicates(inplace = True)
len(data)

12786

Data preprocesing

In [105]:
def cleaning_words(words):
  words = re.sub(r'<[^>]+>', '',words)
  words = re.sub(r'[^a-zA-Z\s]', '',words)
  words.lower()
  return words

def re_stopwords(words):
  stop_words = set(stopwords.words('english'))
  words1 = words.split()
  words = [word for word in words if word not in stop_words]
  return " ".join(words1)


In [106]:
data['new_statement']= data['statement'].astype(str).apply(cleaning_words)
data['new_statement']= data['statement'].astype(str).apply(re_stopwords)
data.head(10)

,label,statement,subject(s),speaker,speaker's job title,state info,party affiliation,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,venue,new_statement
0,TRUE,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview,Building a wall on the U.S.-Mexico border will...
1,FALSE,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference,Wisconsin is on pace to double the number of l...
2,FALSE,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.,Says John McCain has done nothing to help the ...
3,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show,Suzanne Bonamici supports a plan that will cut...
4,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video,When asked by a reporter whether hes at the ce...
5,TRUE,Over the past five years the federal governmen...,"federal-budget,pensions,retirement",brendan-doherty,NaN,Rhode Island,republican,1,2,1,1,0,a campaign website,Over the past five years the federal governmen...
6,TRUE,Says that Tennessee law requires that schools ...,"county-budget,county-government,education,taxes",stand-children-tennessee,Child and education advocacy organization.,Tennessee,none,0,0,0,0,0,in a post on Facebook.,Says that Tennessee law requires that schools ...
7,barely-true,"Says Vice President Joe Biden ""admits that the...","economy,stimulus",john-boehner,Speaker of the House of Representatives,Ohio,republican,13,22,11,4,2,a press release.,"Says Vice President Joe Biden ""admits that the..."
8,TRUE,Donald Trump is against marriage equality. He ...,"gays-and-lesbians,marriage",sean-patrick-maloney,Congressman for NY-18,New York,democrat,0,0,0,0,0,a speech at the Democratic National Convention,Donald Trump is against marriage equality. He ...
9,barely-true,We know that more than half of Hillary Clinton...,foreign-policy,mike-pence,Governor,Indiana,republican,8,10,12,5,0,"comments on ""Meet the Press""",We know that more than half of Hillary Clinton...


In [107]:
data['label'] = data['label'].replace(['half-true', 'mostly-true', 'barely-true', 'pants-fire'], 'FALSE')
print(data['label'].value_counts())

label
FALSE    10733
TRUE      2053
Name: count, dtype: int64


In [108]:
X = data['new_statement']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [109]:
sentences_train = [text.split() for text in X_train]
sentences_test = [text.split() for text in X_test]
full_sentences = sentences_train + sentences_test
word2vec_model = Word2Vec(full_sentences, vector_size=100, window=5, min_count=1, workers=4)

In [110]:
def get_average_word2vec(text):
    words = text.split()
    vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)

In [111]:
X_train_word2vec = np.array([get_average_word2vec(text) for text in X_train])
X_test_word2vec = np.array([get_average_word2vec(text) for text in X_test])
y_train = np.array(y_train.replace({'TRUE': 1, 'FALSE': 0}))
y_test = np.array(y_test.replace({'TRUE': 1, 'FALSE': 0}))
print(X_train_word2vec.shape)
print(y_train.shape)

(10228, 100)
(10228,)


<ipython-input-111-eb9bf5c93888>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = np.array(y_train.replace({'TRUE': 1, 'FALSE': 0}))
<ipython-input-111-eb9bf5c93888>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_test = np.array(y_test.replace({'TRUE': 1, 'FALSE': 0}))


In [112]:
random_forest =  RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train_word2vec, y_train)
y_pred = random_forest.predict(X_test_word2vec)

In [113]:
rf_accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
print(f"Accuracy: {rf_accuracy}")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[2098   12]
 [ 441    7]]
Accuracy: 0.8229085222830336
              precision    recall  f1-score   support

           0       0.83      0.99      0.90      2110
           1       0.37      0.02      0.03       448

    accuracy                           0.82      2558
   macro avg       0.60      0.50      0.47      2558
weighted avg       0.75      0.82      0.75      2558



In [114]:
log_reg = LogisticRegression()
log_reg.fit(X_train_word2vec, y_train)
y_pred = log_reg.predict(X_test_word2vec)

In [115]:
log_reg_accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
print(f"Accuracy: {log_reg_accuracy}")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[2110    0]
 [ 448    0]]
Accuracy: 0.8248631743549648
              precision    recall  f1-score   support

           0       0.82      1.00      0.90      2110
           1       0.00      0.00      0.00       448

    accuracy                           0.82      2558
   macro avg       0.41      0.50      0.45      2558
weighted avg       0.68      0.82      0.75      2558



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [116]:

gbc = GradientBoostingClassifier(n_estimators=100, random_state=42)
gbc.fit(X_train_word2vec, y_train)
y_pred = gbc.predict(X_test_word2vec)

In [117]:
gbc_accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
print(f"Accuracy: {gbc_accuracy}")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[2101    9]
 [ 443    5]]
Accuracy: 0.8232994526974199
              precision    recall  f1-score   support

           0       0.83      1.00      0.90      2110
           1       0.36      0.01      0.02       448

    accuracy                           0.82      2558
   macro avg       0.59      0.50      0.46      2558
weighted avg       0.74      0.82      0.75      2558



In [118]:
dl_model = Sequential([
    Dense(64, activation='relu', input_shape=(100,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [119]:
compile_model = dl_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
dl_model.fit(X_train_word2vec, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose  = 1)

Epoch 1/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8425 - loss: 0.4749 - val_accuracy: 0.8358 - val_loss: 0.4511
Epoch 2/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8413 - loss: 0.4421 - val_accuracy: 0.8358 - val_loss: 0.4497
Epoch 3/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8385 - loss: 0.4442 - val_accuracy: 0.8358 - val_loss: 0.4501
Epoch 4/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8486 - loss: 0.4279 - val_accuracy: 0.8358 - val_loss: 0.4483
Epoch 5/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8439 - loss: 0.4352 - val_accuracy: 0.8358 - val_loss: 0.4469
Epoch 6/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8455 - loss: 0.4312 - val_accuracy: 0.8358 - val_loss: 0.4455
Epoch 7/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8460 - loss: 0.4279 - val_accuracy: 0.8358 - val_loss: 0.4451
Epoch 8/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8413 - loss: 0.4350 - val_accuracy: 0.

In [120]:
test_loss, test_accuracy = dl_model.evaluate(X_test_word2vec, y_test)
print(f"Test Accuracy: {test_accuracy}")

80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8223 - loss: 0.4681
Test Accuracy: 0.8248631954193115
